In [1]:
import numpy as np
seed = 7
np.random.seed(seed)

In [2]:
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import np_utils

Using Theano backend.


In [3]:
from keras import optimizers

In [4]:
def create_model8():
    model8 = Sequential()
    model8.add(Dense(10, input_dim=9, activation='relu'))
    model8.add(Dense(10, activation='relu'))
    model8.add(Dense(10, activation='relu'))
    model8.add(Dense(10, activation='relu'))
    model8.add(Dense(10, activation='relu'))
    model8.add(Dense(10, activation='relu'))
    model8.add(Dense(10, activation='relu'))
    model8.add(Dense(10, activation='relu'))
    model8.add(Dense(1, activation='sigmoid'))
    adm = optimizers.Adam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
    model8.compile(loss='binary_crossentropy', optimizer=adm, metrics=['accuracy'])
    return model8

In [5]:
from keras.models import load_model

In [6]:
model=create_model8()

In [ ]:
model.save_weights('my_model_weights.h5')

In [7]:
class_weight = [{0 : 1., 1: 2},
                {0 : 1., 1: 4},
                {0 : 1., 1: 8},
                {0 : 1., 1: 16},
                {0 : 1., 1: 32},
                {0 : 1., 1: 64}]

In [8]:
epochs = [10, 20, 50, 100, 200, 500]

In [9]:
param_gridNN = dict(epochs=epochs, class_weight=class_weight )

In [10]:
from sklearn.metrics import confusion_matrix

In [11]:
modelNN = KerasClassifier(build_fn=create_model8, verbose=0)

In [12]:
import pandas as pd

In [13]:
dfA = pd.read_csv("wbc.csv")
dfA=dfA.replace('?',np.nan)
dfA['bare_nuclei'] = pd.to_numeric(dfA.bare_nuclei.fillna(dfA.bare_nuclei.median()))

In [14]:
X = dfA.drop([dfA.columns[-1]], axis=1)
y = dfA.jenis.map({2: 0, 4: 1})

In [15]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.33, random_state=seed )

In [16]:
from sklearn.metrics import confusion_matrix

In [17]:
def cost_breast_cancer(y_true, y_pred):
    CM = confusion_matrix(y_true, y_pred)
    FN = CM[1][0]
    FP = CM[0][1]
    costBC=((-1*((228.35*FP)+(2850000*FN))))
    return costBC

In [18]:
def output_score(y_true, y_pred):
    CM = confusion_matrix(y_true, y_pred)
    TN = CM[0][0]
    FN = CM[1][0]
    TP = CM[1][1]
    FP = CM[0][1]
    specificity=(TN/(FP+TN))
    sensitivity=(TP/(TP+FN))
    accuracy=((TP+TN)/(TP+FP+TN+FN))
    costBC=((-1*((228.35*FP)+(2850000*FN))))
    return (specificity, sensitivity, accuracy, costBC)

In [19]:
def specificity(y_true, y_pred):
    CM = confusion_matrix(y_true, y_pred)
    TN = CM[0][0]
    FN = CM[1][0]
    TP = CM[1][1]
    FP = CM[0][1]
    nilai=(TN/(FP+TN))
    return nilai

In [20]:
from sklearn.metrics import make_scorer
my_custom_scorer=make_scorer(cost_breast_cancer, greater_is_better=True)
score_specificity=make_scorer(specificity, greater_is_better=True)
output_scoring=make_scorer(output_score, greater_is_better=True)

In [21]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
from __future__ import print_function
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_curve, roc_auc_score

In [23]:
print(__doc__)


scores = ['recall']

for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()

    clf3 = GridSearchCV(estimator=modelNN, param_grid=param_gridNN, cv=10, 
                       scoring='recall')
    clf3.fit(X_train.as_matrix(), y_train)

    print("Best parameters set found on development set:")
    print()
    print(clf3.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means3 = clf3.cv_results_['mean_test_score']
    stds3 = clf3.cv_results_['std_test_score']
    for mean, std, params in zip(means3, stds3, clf3.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std, params))
    print()
    print("Detailed classification report:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    y_true, y_pred = y_test, clf3.predict(X_test.as_matrix())
    print(classification_report(y_true, y_pred))
    print()
    print("Specificity, Sensitivity, Accuracy:")
    output_score(y_true, y_pred)
    print()

Automatically created module for IPython interactive environment
# Tuning hyper-parameters for recall

Best parameters set found on development set:

{'epochs': 20, 'class_weight': {0: 1.0, 1: 8}}

Grid scores on development set:

0.988 (+/-0.025) for {'epochs': 10, 'class_weight': {0: 1.0, 1: 2}}
0.987 (+/-0.026) for {'epochs': 20, 'class_weight': {0: 1.0, 1: 2}}
0.974 (+/-0.044) for {'epochs': 50, 'class_weight': {0: 1.0, 1: 2}}
0.975 (+/-0.043) for {'epochs': 100, 'class_weight': {0: 1.0, 1: 2}}
0.967 (+/-0.046) for {'epochs': 200, 'class_weight': {0: 1.0, 1: 2}}
0.962 (+/-0.054) for {'epochs': 500, 'class_weight': {0: 1.0, 1: 2}}
0.994 (+/-0.018) for {'epochs': 10, 'class_weight': {0: 1.0, 1: 4}}
0.993 (+/-0.020) for {'epochs': 20, 'class_weight': {0: 1.0, 1: 4}}
0.979 (+/-0.043) for {'epochs': 50, 'class_weight': {0: 1.0, 1: 4}}
0.989 (+/-0.023) for {'epochs': 100, 'class_weight': {0: 1.0, 1: 4}}
0.987 (+/-0.040) for {'epochs': 200, 'class_weight': {0: 1.0, 1: 4}}
0.986 (+/-0.028)

In [24]:
output_score(y_true, y_pred)

(0.95394736842105265,
 0.97468354430379744,
 0.96103896103896103,
 -5701598.4500000002)

In [25]:
 model.load_weights('my_model_weights.h5')

In [26]:
best_class_weight = {0: 1.0, 1: 8}

In [27]:
bestmodelNN = KerasClassifier(build_fn=create_model8, epochs= 20, class_weight = best_class_weight, verbose=0)

In [28]:
bestmodelNN.fit(X_train.as_matrix(), y_train )

In [18]:
import warnings
warnings.filterwarnings('ignore', message='.*is ill-defined', append=True)
warnings.filterwarnings('ignore', category=DeprecationWarning)

In [23]:
scores4=cross_val_score(bestmodelNN,X_train.as_matrix(), y_train, cv=10, scoring=my_custom_scorer)
print("Score cost: %0.4f (+/- %0.4f)" % (scores4.mean(), scores4.std() * 2))

Score cost: -342.5250 (+/- 549.9396)


In [ ]:
scores4

In [30]:
model.load_weights('my_model_weights.h5')
scores1=cross_val_score(bestmodelNN,X_train.as_matrix(),y_train,cv=10,scoring=score_specificity)
print("Specificity: %0.4f (+/- %0.4f)" % (scores1.mean(), scores1.std() * 2))
model.load_weights('my_model_weights.h5')
scores2=cross_val_score(bestmodelNN,X_train.as_matrix(),y_train,cv=10,scoring='accuracy')
print("Accuracy: %0.4f (+/- %0.4f)" % (scores2.mean(), scores2.std() * 2))
model.load_weights('my_model_weights.h5')
scores3=cross_val_score(bestmodelNN,X_train.as_matrix(),y_train,cv=10,scoring='recall')
print("Recall: %0.4f (+/- %0.4f)" % (scores3.mean(), scores3.std() * 2))
model.load_weights('my_model_weights.h5')
scores4=cross_val_score(bestmodelNN,X_train.as_matrix(), y_train, cv=10, scoring=my_custom_scorer)
print("Score cost: %0.4f (+/- %0.4f)" % (scores4.mean(), scores4.std() * 2))

Specificity: 0.9593 (+/- 0.0850)
Accuracy: 0.9742 (+/- 0.0464)
Recall: 0.9882 (+/- 0.0706)
Score cost: -570411.0300 (+/- 2279817.4847)


In [31]:
def cost_breast_cancer2(y_true, y_pred):
    CM = confusion_matrix(y_true, y_pred)
    FN = CM[1][0]
    FP = CM[0][1]
    costBC=((-1*((228.35*FP)+(28500*FN))))
    return costBC

In [32]:
my_custom_scorer2=make_scorer(cost_breast_cancer2, greater_is_better=True)

In [33]:
print(__doc__)

model.load_weights('my_model_weights.h5')
scores = [my_custom_scorer2]

for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()

    clf4 = GridSearchCV(estimator=modelNN, param_grid=param_gridNN, cv=10, 
                       scoring=my_custom_scorer2)
    clf4.fit(X_train.as_matrix(), y_train)

    print("Best parameters set found on development set:")
    print()
    print(clf4.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means4 = clf4.cv_results_['mean_test_score']
    stds4 = clf4.cv_results_['std_test_score']
    for mean, std, params in zip(means4, stds4, clf4.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std, params))
    print()
    print("Detailed classification report:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    y_true, y_pred = y_test, clf4.predict(X_test.as_matrix())
    print(classification_report(y_true, y_pred))
    print()
    print("Specificity, Sensitivity, Accuracy:")
    output_score(y_true, y_pred)
    print()

Automatically created module for IPython interactive environment
# Tuning hyper-parameters for make_scorer(cost_breast_cancer2)

Best parameters set found on development set:

{'epochs': 50, 'class_weight': {0: 1.0, 1: 16}}

Grid scores on development set:

-14333.623 (+/-19029.140) for {'epochs': 10, 'class_weight': {0: 1.0, 1: 2}}
-11553.809 (+/-14086.913) for {'epochs': 20, 'class_weight': {0: 1.0, 1: 2}}
-11593.238 (+/-18833.031) for {'epochs': 50, 'class_weight': {0: 1.0, 1: 2}}
-65866.910 (+/-141628.263) for {'epochs': 100, 'class_weight': {0: 1.0, 1: 2}}
-6019.067 (+/-11331.030) for {'epochs': 200, 'class_weight': {0: 1.0, 1: 2}}
-12409.447 (+/-26006.586) for {'epochs': 500, 'class_weight': {0: 1.0, 1: 2}}
-3226.661 (+/-8449.218) for {'epochs': 10, 'class_weight': {0: 1.0, 1: 4}}
-3156.888 (+/-8472.702) for {'epochs': 20, 'class_weight': {0: 1.0, 1: 4}}
-11662.036 (+/-18860.082) for {'epochs': 50, 'class_weight': {0: 1.0, 1: 4}}
-14416.477 (+/-19170.913) for {'epochs': 100, 'cla

In [34]:
print("Best score obtained: {0}".format(clf4.best_score_))
print("Parameters:")
for key, value in clf4.best_params_.items():
    print("\t{}: {}".format(key, value))

Best score obtained: -250.30673076923077
Parameters:
	epochs: 50
	class_weight: {0: 1.0, 1: 16}


In [35]:
best_class_weight = {0: 1.0, 1: 16}

In [36]:
bestmodelNN11 = KerasClassifier(build_fn=create_model8, epochs= 50, class_weight = best_class_weight, verbose=0)

In [37]:
model.load_weights('my_model_weights.h5')
bestmodelNN11.fit(X_train.as_matrix(), y_train )

In [38]:
import warnings
warnings.filterwarnings('ignore', message='.*is ill-defined', append=True)
warnings.filterwarnings('ignore', category=DeprecationWarning)
model.load_weights('my_model_weights.h5')
scores1=cross_val_score(bestmodelNN11,X_train.as_matrix(),y_train,cv=10,scoring=score_specificity)
print("Specificity: %0.4f (+/- %0.4f)" % (scores1.mean(), scores1.std() * 2))
model.load_weights('my_model_weights.h5')
scores2=cross_val_score(bestmodelNN11,X_train.as_matrix(),y_train,cv=10,scoring='accuracy')
print("Accuracy: %0.4f (+/- %0.4f)" % (scores2.mean(), scores2.std() * 2))
model.load_weights('my_model_weights.h5')
scores3=cross_val_score(bestmodelNN11,X_train.as_matrix(),y_train,cv=10,scoring='recall')
print("Recall: %0.4f (+/- %0.4f)" % (scores3.mean(), scores3.std() * 2))
model.load_weights('my_model_weights.h5')
scores4=cross_val_score(bestmodelNN11,X_train.as_matrix(), y_train, cv=10, scoring=my_custom_scorer)
print("Score cost: %0.4f (+/- %0.4f)" % (scores4.mean(), scores4.std() * 2))

Specificity: 0.9553 (+/- 0.0795)
Accuracy: 0.9353 (+/- 0.1800)
Recall: 0.9933 (+/- 0.0400)
Score cost: -570251.1850 (+/- 2279863.0266)


In [ ]:
for train in kfold.split(X_train, y_train):
  # create model
    model = build_model()
    model.load_weights('my_model_weights.h5')
    model.fit(X_train.as_matrix(), y_train, epochs=10, class_weight = {0: 1.0, 1: 8}, verbose=0)

    scores = model.evaluate(X_test.as_matrix(), y_test)
    print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    print("%s: %.2f%%" % (model.metrics_names[2], scores[2]*100))
    print("%s: %.2f%%" % (model.metrics_names[3], scores[3]*100))
    print()
    print()
    
    cvscores.append(scores[1] * 100)
    cvscores2.append(scores[2] * 100)
    cvscores3.append(scores[3] * 100)

print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores2), np.std(cvscores2)))   
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores3), np.std(cvscores3))) 

In [24]:
epochs = [10, 20, 50, 100, 200, 500]
class_weight = [{0 : 1., 1: 2},
                {0 : 1., 1: 4},
                {0 : 1., 1: 8},
                {0 : 1., 1: 16},
                {0 : 1., 1: 32},
                {0 : 1., 1: 64}]
results=[]
model.load_weights('my_model_weights.h5')

for epochs_idx in range(len(epochs)):
  for class_weight_idx in range(len(class_weight)):
    param1 = epochs[epochs_idx]
    param2 = class_weight[class_weight_idx]
    bestmodelNN = KerasClassifier(build_fn=create_model8, epochs= param1, class_weight = param2, verbose=0)
    model.load_weights('my_model_weights.h5')
    bestmodelNN.fit(X_train.as_matrix(), y_train )
    print("Epochs : %r & Class Weight : %r)" % (param1, param2))
    scores1=cross_val_score(bestmodelNN,X_train.as_matrix(),y_train,cv=10,scoring=score_specificity)
    print("Specificity: %0.4f (+/- %0.4f)" % (scores1.mean(), scores1.std() ))
    model.load_weights('my_model_weights.h5')
    scores2=cross_val_score(bestmodelNN,X_train.as_matrix(),y_train,cv=10,scoring='accuracy')
    print("Accuracy: %0.4f (+/- %0.4f)" % (scores2.mean(), scores2.std() ))
    model.load_weights('my_model_weights.h5')
    scores3=cross_val_score(bestmodelNN,X_train.as_matrix(),y_train,cv=10,scoring='recall')
    print("Recall: %0.4f (+/- %0.4f)" % (scores3.mean(), scores3.std() ))
    model.load_weights('my_model_weights.h5')
    scores4=cross_val_score(bestmodelNN,X_train.as_matrix(), y_train, cv=10, scoring=my_custom_scorer)
    print("Score cost: %0.4f (+/- %0.4f)" % (scores4.mean(), scores4.std() ))
    print()
    print()


Epochs : 10 & Class Weight : {0: 1.0, 1: 2})
Specificity: 0.9704 (+/- 0.0301)
Accuracy: 0.9636 (+/- 0.0290)
Recall: 0.9724 (+/- 0.0465)
Score cost: -855228.3500 (+/- 1305934.4371)


Epochs : 10 & Class Weight : {0: 1.0, 1: 4})
Specificity: 0.9670 (+/- 0.0324)
Accuracy: 0.9700 (+/- 0.0198)
Recall: 0.9941 (+/- 0.0176)
Score cost: -285274.0200 (+/- 854984.8180)


Epochs : 10 & Class Weight : {0: 1.0, 1: 8})
Specificity: 0.9543 (+/- 0.0340)
Accuracy: 0.9593 (+/- 0.0296)
Recall: 0.9933 (+/- 0.0200)
Score cost: -855570.8750 (+/- 1305859.8090)


Epochs : 10 & Class Weight : {0: 1.0, 1: 16})
Specificity: 0.9241 (+/- 0.0630)
Accuracy: 0.9423 (+/- 0.0712)
Recall: 0.9867 (+/- 0.0400)
Score cost: -1004.7400 (+/- 1105.5536)


Epochs : 10 & Class Weight : {0: 1.0, 1: 32})
Specificity: 0.8487 (+/- 0.1251)
Accuracy: 0.8883 (+/- 0.0884)
Recall: 0.9933 (+/- 0.0200)
Score cost: -285913.4000 (+/- 854771.9733)


Epochs : 10 & Class Weight : {0: 1.0, 1: 64})
Specificity: 0.7890 (+/- 0.2770)
Accuracy: 0.8847

In [22]:
best_class_weight = {0: 1.0, 1: 8}

In [23]:
bestmodelNN_test = KerasClassifier(build_fn=create_model8, epochs= 200, class_weight = best_class_weight, verbose=0)

In [26]:
import warnings
warnings.filterwarnings('ignore', message='.*is ill-defined', append=True)
warnings.filterwarnings('ignore', category=DeprecationWarning)
for i in range(10):
    model.load_weights('my_model_weights.h5')
    scores1=cross_val_score(bestmodelNN_test, X_train.as_matrix(),y_train,cv=10,scoring=score_specificity)
    print("Specificity: %0.4f (+/- %0.4f)" % (scores1.mean(), scores1.std()*2))
    model.load_weights('my_model_weights.h5')
    scores3=cross_val_score(bestmodelNN_test, X_train.as_matrix(),y_train,cv=10,scoring='recall')
    print("Recall: %0.4f (+/- %0.4f)" % (scores3.mean(), scores3.std()*2))
    print()
    print()

Specificity: 0.9707 (+/- 0.0545)
Recall: 0.9736 (+/- 0.0881)


Specificity: 0.9602 (+/- 0.0770)
Recall: 0.9862 (+/- 0.0553)


Specificity: 0.9669 (+/- 0.0585)
Recall: 0.9941 (+/- 0.0353)


Specificity: 0.9526 (+/- 0.0703)
Recall: 0.9941 (+/- 0.0353)


Specificity: 0.9654 (+/- 0.0615)
Recall: 0.9748 (+/- 0.0867)


Specificity: 0.9664 (+/- 0.0504)
Recall: 0.9814 (+/- 0.0578)


Specificity: 0.9603 (+/- 0.0590)
Recall: 0.9748 (+/- 0.0629)


Specificity: 0.9706 (+/- 0.0616)
Recall: 0.9933 (+/- 0.0400)


Specificity: 0.9179 (+/- 0.2974)
Recall: 0.9800 (+/- 0.0854)


Specificity: 0.9576 (+/- 0.0843)
Recall: 0.9862 (+/- 0.0553)


